In [6]:
import pandas as pd

#    read CSV file created by R notebook
df = pd.read_csv("Minimum Wage Data_2022.csv")
df.rename(columns={'Department.Of.Labor.Cleaned.Low.Value': 'Department_Of_Labor_Cleaned_Low_Value',
                   'Federal.Minimum.Wage': 'Federal_Minimum_Wage', 'Federal.Minimum.Wage.2022.Dollars':'Federal_Minimum_Wage_2022_Dollars',
                    'CPI.Average':'CPI_Average'},
          inplace=True)
df.head()

,Year,State,Department_Of_Labor_Cleaned_Low_Value,Federal_Minimum_Wage,Federal_Minimum_Wage_2022_Dollars,Effective.Minimum.Wage,Effective.Minimum.Wage.2022.Dollars,CPI_Average,Department.Of.Labor.Uncleaned.Data,Department.Of.Labor.Cleaned.Low.Value.2022.Dollars,Department.Of.Labor.Cleaned.High.Value,Department.Of.Labor.Cleaned.High.Value.2022.Dollars,Footnote
0,1968,Alabama,0.00000,1.15,9.53,1.15,9.53,34.8,...,0.00,0.00000,0.00,NaN
1,1968,Alaska,2.10000,1.15,9.53,2.10,17.40,34.8,2.1,17.40,2.10000,17.40,NaN
2,1968,Arizona,0.46800,1.15,9.53,1.15,9.53,34.8,18.72 - 26.40/wk(b),3.88,0.66000,5.47,(b)
3,1968,Arkansas,0.15625,1.15,9.53,1.15,9.53,34.8,1.25/day(b),1.29,0.15625,1.29,(b)
4,1968,California,1.65000,1.15,9.53,1.65,13.67,34.8,1.65(b),13.67,1.65000,13.67,(b)


In [4]:
#    acquiring FIPS data
import pandas as pd
import numpy as np
from urllib import request

#    define the remote file to retrieve, and the local file name
remote_url = 'https://www2.census.gov/geo/docs/reference/state.txt'
local_file = "ansi_state_fips"    
#    download remote and save locally
request.urlretrieve(remote_url, local_file)
#    read into df (this file has |)
fips = pd.read_csv("ansi_state_fips", sep= "|")
fips.rename(columns={'STATE': 'id', 'STATE_NAME': 'State'}, inplace=True)
fips_to_merge=fips[['State', 'id']]
fips_to_merge.head()

,State,id
0,Alabama,1
1,Alaska,2
2,Arizona,4
3,Arkansas,5
4,California,6


In [28]:
#merge FIPS id with wage data
merged=df.merge(fips_to_merge, how='left', on='State')
merged.rename(columns={'Department_Of_Labor_Cleaned_Low_Value': 'State_Minimum_Wage'}, inplace=True)
merged.head()

,Year,State,State_Minimum_Wage,Federal_Minimum_Wage,Federal_Minimum_Wage_2022_Dollars,Effective.Minimum.Wage,Effective.Minimum.Wage.2022.Dollars,CPI_Average,Department.Of.Labor.Uncleaned.Data,Department.Of.Labor.Cleaned.Low.Value.2022.Dollars,Department.Of.Labor.Cleaned.High.Value,Department.Of.Labor.Cleaned.High.Value.2022.Dollars,Footnote,id
0,1968,Alabama,0.00000,1.15,9.53,1.15,9.53,34.8,...,0.00,0.00000,0.00,NaN,1
1,1968,Alaska,2.10000,1.15,9.53,2.10,17.40,34.8,2.1,17.40,2.10000,17.40,NaN,2
2,1968,Arizona,0.46800,1.15,9.53,1.15,9.53,34.8,18.72 - 26.40/wk(b),3.88,0.66000,5.47,(b),4
3,1968,Arkansas,0.15625,1.15,9.53,1.15,9.53,34.8,1.25/day(b),1.29,0.15625,1.29,(b),5
4,1968,California,1.65000,1.15,9.53,1.65,13.67,34.8,1.65(b),13.67,1.65000,13.67,(b),6


In [36]:
df_for_maps=merged[['State','id', 'Year', 'State_Minimum_Wage', 'Federal_Minimum_Wage', 'Federal_Minimum_Wage_2022_Dollars', 'CPI_Average']]
df_for_maps.head()


,State,id,Year,State_Minimum_Wage,Federal_Minimum_Wage,Federal_Minimum_Wage_2022_Dollars,CPI_Average
0,Alabama,1,1968,0.00000,1.15,9.53,34.8
1,Alaska,2,1968,2.10000,1.15,9.53,34.8
2,Arizona,4,1968,0.46800,1.15,9.53,34.8
3,Arkansas,5,1968,0.15625,1.15,9.53,34.8
4,California,6,1968,1.65000,1.15,9.53,34.8


In [37]:
# calculate the effective minimum wage (greater of state or fed min wage)
df_for_maps["Effective_Minimum_Wage"] = df_for_maps.loc[:, ("State_Minimum_Wage", "Federal_Minimum_Wage")].max(axis=1)
df_for_maps.head()


/var/folders/jj/5nzz4q4n3ml1wy7zbrgyl5k80000gn/T/ipykernel_3153/3394782584.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_for_maps["Effective_Minimum_Wage"] = df_for_maps.loc[:, ("State_Minimum_Wage", "Federal_Minimum_Wage")].max(axis=1)


,State,id,Year,State_Minimum_Wage,Federal_Minimum_Wage,Federal_Minimum_Wage_2022_Dollars,CPI_Average,Effective_Minimum_Wage
0,Alabama,1,1968,0.00000,1.15,9.53,34.8,1.15
1,Alaska,2,1968,2.10000,1.15,9.53,34.8,2.10
2,Arizona,4,1968,0.46800,1.15,9.53,34.8,1.15
3,Arkansas,5,1968,0.15625,1.15,9.53,34.8,1.15
4,California,6,1968,1.65000,1.15,9.53,34.8,1.65


In [38]:
## AUTHOR: JOE LISLE
## https://github.com/Lislejoem/Minimum-Wage-by-State-1968-to-2020/blob/master/2012%20-%20Minimum%20Wage%20by%20State.Rmd
## It's a fairly simple calculation to calculate 2018 dollars from any point in time!
## Just create an equation (x/y) * b, with the x being the average Consumer Price Index (CPI) from 2018 and y being the average CPI from the other year in the denominator. B is equal to the dollar amount in y's year
# MT translation: ((current year CPI)/ (other year CPI)) * (other year actual $)
# 2022 CPI
df_for_maps.tail()


,State,id,Year,State_Minimum_Wage,Federal_Minimum_Wage,Federal_Minimum_Wage_2022_Dollars,CPI_Average,Effective_Minimum_Wage
2965,Virginia,51,2022,11.00,7.25,7.25,288.347,11.00
2966,Washington,53,2022,14.49,7.25,7.25,288.347,14.49
2967,West Virginia,54,2022,8.75,7.25,7.25,288.347,8.75
2968,Wisconsin,55,2022,7.25,7.25,7.25,288.347,7.25
2969,Wyoming,56,2022,7.25,7.25,7.25,288.347,7.25


In [39]:
# 2022 CPI = 288.347
# formula would be (288.347/df['CPI_Average'])*df['Effective_Min_Wage']
df_for_maps["Effective_Minimum_Wage_2022_Dollars"] = round((288.347 / df_for_maps['CPI_Average']) * df_for_maps['Effective_Minimum_Wage'], 2)
df_for_maps.head()


/var/folders/jj/5nzz4q4n3ml1wy7zbrgyl5k80000gn/T/ipykernel_3153/3668929013.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_for_maps["Effective_Minimum_Wage_2022_Dollars"] = round((288.347 / df_for_maps['CPI_Average']) * df_for_maps['Effective_Minimum_Wage'], 2)


,State,id,Year,State_Minimum_Wage,Federal_Minimum_Wage,Federal_Minimum_Wage_2022_Dollars,CPI_Average,Effective_Minimum_Wage,Effective_Minimum_Wage_2022_Dollars
0,Alabama,1,1968,0.00000,1.15,9.53,34.8,1.15,9.53
1,Alaska,2,1968,2.10000,1.15,9.53,34.8,2.10,17.40
2,Arizona,4,1968,0.46800,1.15,9.53,34.8,1.15,9.53
3,Arkansas,5,1968,0.15625,1.15,9.53,34.8,1.15,9.53
4,California,6,1968,1.65000,1.15,9.53,34.8,1.65,13.67


In [40]:
# and export only the columns each visualization will need
state_actual = df_for_maps[['State','id', 'Year', 'State_Minimum_Wage', 'Federal_Minimum_Wage', 'CPI_Average']]
                            
effective_2022_dollars = df_for_maps[['State','id', 'Year', 'Federal_Minimum_Wage', 'Federal_Minimum_Wage_2022_Dollars', 
                                      'Effective_Minimum_Wage', 'Effective_Minimum_Wage_2022_Dollars', 'CPI_Average']]
state_actual.head()

,State,id,Year,State_Minimum_Wage,Federal_Minimum_Wage,CPI_Average
0,Alabama,1,1968,0.00000,1.15,34.8
1,Alaska,2,1968,2.10000,1.15,34.8
2,Arizona,4,1968,0.46800,1.15,34.8
3,Arkansas,5,1968,0.15625,1.15,34.8
4,California,6,1968,1.65000,1.15,34.8


In [41]:
# let's check the other one too
effective_2022_dollars.head()

,State,id,Year,Federal_Minimum_Wage,Federal_Minimum_Wage_2022_Dollars,Effective_Minimum_Wage,Effective_Minimum_Wage_2022_Dollars,CPI_Average
0,Alabama,1,1968,1.15,9.53,1.15,9.53,34.8
1,Alaska,2,1968,1.15,9.53,2.10,17.40,34.8
2,Arizona,4,1968,1.15,9.53,1.15,9.53,34.8
3,Arkansas,5,1968,1.15,9.53,1.15,9.53,34.8
4,California,6,1968,1.15,9.53,1.65,13.67,34.8


In [42]:
#and save to CSV!
state_actual.to_csv('State_Actual.csv')
effective_2022_dollars.to_csv('Effective_Rate_2022_Dollars.csv')